# 기출3

In [2]:
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings('ignore')

In [3]:
dat=pd.read_csv('test/Online2.txt', sep='\t', encoding='UTF-8')
dat.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom


In [4]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null object
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [3]:
dat.InvoiceNo.dtype

dtype('O')

In [5]:
dat.InvoiceNo.isna().sum()

0

# 전처리

In [6]:
pre1=dat[dat.InvoiceNo.str.isnumeric()]

In [7]:
pre2=pre1[(pre1['UnitPrice'] > 0)  &  (pre1['Quantity'] > 0)]

In [8]:
pre2['TotOrder'] = pre2['Quantity'] * pre2['UnitPrice']

In [9]:
basetable1=pre2.copy()

In [10]:
len(basetable1)

530103

# Q1

In [26]:
basetable1['InvoiceNo'].drop_duplicates()

0         536365
7         536366
9         536367
21        536368
25        536369
           ...  
541865    581583
541867    581584
541869    581585
541890    581586
541894    581587
Name: InvoiceNo, Length: 19959, dtype: object

In [27]:
q1=basetable1.groupby(['InvoiceNo', 'Country'])['TotOrder'].sum()
q1=q1.reset_index(drop=False)
len(q1)

19959

In [28]:
q1=q1.rename(columns={'TotOrder':'InvTotOrder'})

In [29]:
q1

,InvoiceNo,Country,InvTotOrder
0,536365,United Kingdom,139.12
1,536366,United Kingdom,22.20
2,536367,United Kingdom,278.73
3,536368,United Kingdom,70.05
4,536369,United Kingdom,17.85
...,...,...,...
19954,581583,United Kingdom,124.60
19955,581584,United Kingdom,140.64
19956,581585,United Kingdom,329.05
19957,581586,United Kingdom,339.20


In [30]:
a1=basetable1['Country'].value_counts()

In [31]:
sel_list=a1[(a1 >= 50) & (a1 <= 400)].index
sel_list

Index(['Austria', 'Denmark', 'Poland', 'Japan', 'Israel', 'Hong Kong',
       'Singapore', 'Iceland', 'USA', 'Canada', 'Greece', 'Malta',
       'United Arab Emirates', 'European Community', 'RSA'],
      dtype='object')

In [32]:
len(sel_list)

15

In [33]:
q1=q1[q1['Country'].isin(sel_list)]

In [34]:
q1

,InvoiceNo,Country,InvTotOrder
306,536971,Poland,248.16
417,537218,Japan,320.08
581,537626,Iceland,711.79
746,537899,Japan,3794.40
757,538003,Denmark,1281.50
...,...,...,...
19458,580553,USA,615.28
19558,580716,Denmark,168.90
19587,580752,Greece,680.99
19779,581180,Iceland,224.82


In [35]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [37]:
aov=ols('InvTotOrder~Country', q1).fit()
q1_out=anova_lm(aov)
q1_out

,df,sum_sq,mean_sq,F,PR(>F)
Country,14.0,6.414030e+07,4.581450e+06,3.726267,0.000035
Residual,120.0,1.475402e+08,1.229501e+06,NaN,NaN


In [39]:
q1_out['F']['Country']

3.7262665618641413

# Q2

In [40]:
q2=basetable1.groupby(['InvoiceDate', 'InvoiceNo'])['TotOrder'].sum()
q2=q2.reset_index(drop=False)
q2

,InvoiceDate,InvoiceNo,TotOrder
0,2010-12-01,536365,139.12
1,2010-12-01,536366,22.20
2,2010-12-01,536367,278.73
3,2010-12-01,536368,70.05
4,2010-12-01,536369,17.85
...,...,...,...
19954,2011-12-09,581583,124.60
19955,2011-12-09,581584,140.64
19956,2011-12-09,581585,329.05
19957,2011-12-09,581586,339.20


In [41]:
q2.dtypes

InvoiceDate     object
InvoiceNo       object
TotOrder       float64
dtype: object

In [47]:
q2=q2.rename(columns={'TotOrder':'InvTotOrder'})
q2.head(2)

,InvoiceDate,InvoiceNo,InvTotOrder,day
0,2010-12-01,536365,139.12,수
1,2010-12-01,536366,22.20,수


In [43]:
import locale
locale.setlocale(locale.LC_ALL,'')

'Korean_Korea.949'

In [44]:
q2['day']=pd.to_datetime(q2['InvoiceDate']).dt.strftime('%a')
q2.head(3)

,InvoiceDate,InvoiceNo,TotOrder,day
0,2010-12-01,536365,139.12,수
1,2010-12-01,536366,22.20,수
2,2010-12-01,536367,278.73,수


In [50]:
q2_out=q2.groupby('day')['InvTotOrder'].apply(
    lambda x: x[x >= x.quantile(0.8)].mean())  # 요일별 상위 20% 평균

q2_out.nsmallest(1)

day
일    942.247937
Name: InvTotOrder, dtype: float64

# Q3

In [72]:
q3_1=basetable1.groupby('StockCode')[['UnitPrice', 'Quantity']]
q3_2=q3_1.agg({'UnitPrice':'mean','Quantity': 'sum'})
q3_2

,UnitPrice,Quantity
StockCode,,
10002,1.086620,860
10080,0.410909,303
10120,0.210000,193
10123C,0.650000,5
10124A,0.420000,16
...,...,...
gift_0001_20,16.708889,10
gift_0001_30,25.075714,7
gift_0001_40,33.566667,3


In [73]:
# q3_1.groups
q3_1.get_group('10002')

,UnitPrice,Quantity
31,0.85,48
142,0.85,12
4272,0.85,1
5466,0.85,1
5546,1.66,5
...,...,...
142354,0.85,120
143465,0.85,4
143930,0.85,2
158070,0.85,62


In [74]:
q3_3=q3_2[(q3_2['UnitPrice'] >= 4) & ( q3_2['Quantity'] >= 600)]
len(q3_2), len(q3_3)

(3921, 274)

In [78]:
# 최종 분석 대상
q3_4=basetable1[basetable1['StockCode'].isin(q3_3.index)]

In [82]:
q3_4

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotOrder
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01,7.65,17850.0,United Kingdom,15.30
...,...,...,...,...,...,...,...,...,...
541900,581587,22730,ALARM CLOCK BAKELIKE IVORY,4,2011-12-09,3.75,12680.0,France,15.00
541903,581587,23256,CHILDRENS CUTLERY SPACEBOY,4,2011-12-09,4.15,12680.0,France,16.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09,4.15,12680.0,France,16.60


In [83]:
# 구매자와 주문날짜 기준으로 물품 목록 생성
a2=q3_4.groupby(['CustomerID','InvoiceDate'])['StockCode']
q3_5=a2.apply(lambda x: x.unique())

In [88]:
#  단일 물품은 제외
q3_6=q3_5[q3_5.apply(len) > 1]

In [90]:
q3_6=q3_6.reset_index(drop=False)

In [91]:
q3_6.CustomerID.isna().sum()

0

In [92]:
# 연관성 분석 수행

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [94]:
te=TransactionEncoder().fit(q3_6['StockCode'])
te_arr = te.transform(q3_6['StockCode'])
te_arr

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [95]:
# 장바구니 생성
te_df=pd.DataFrame(te_arr, columns=te.columns_)

In [96]:
apr=apriori(te_df, min_support=0.01, max_len=2, use_colnames=True)

In [99]:
rules=association_rules(apr, min_threshold=0.01)
# rules

In [98]:
rules[rules['antecedents'] =={'23012'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
176,(23012),(23013),0.016607,0.015152,0.0107,0.64433,42.525773,0.010449,2.768994
